In [17]:
import sys
sys.path.insert(0, "/home/alexp/NBA_Models/src/scripts/")

In [18]:
import transform_db
from transform_db import load_clean
from transform_db import transform
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sqlite3


pd.set_option("display.max_columns",None)

In [19]:
path = "/home/alexp/NBA_Models/sqlite/db/nba_data.db"
load = load_clean(path)
agg_boxscores = load.agg_boxscores_raw()

In [20]:
conn = sqlite3.connect(path)

clean = transform(conn,2013,2023)
data = clean.load_team_data()
data = clean.clean_team_data(data)
data = data.dropna(subset='PCT_PTS_2PT')
data = clean.convert_pcts(data)

In [21]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaled_df = data.loc[:,data.columns[9:]]
scaled_array = scaler.fit_transform(scaled_df)
scaled = pd.DataFrame(scaled_array,columns=scaled_df.columns)

In [22]:
add_columns = data.loc[:,data.columns[:9]]
data[scaled.columns] = scaled.values

# Create Input Array

In [23]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
encoder.fit(data['TEAM_ABBREVIATION'])

data['TEAM_ABBREVIATION'] = encoder.transform(data['TEAM_ABBREVIATION'])

In [24]:
filter = ['SEASON',"TEAM_ID",'TEAM_NAME','GAME_ID','GAME_DATE','MATCHUP']
data_clean = data[data.columns.difference(filter)]

In [25]:
X_list = []
y_list = []
window_size = 10

for i in range(len(data_clean['TEAM_ABBREVIATION'].unique())):
    team = data_clean.loc[data_clean['TEAM_ABBREVIATION'] == i]
    team_label = team['WL'].copy()
    team = team.drop('WL',axis=1)

    team_np = team.to_numpy()
    team_label = team_label.to_numpy()

    #temp_X = []
    #temp_y = []
    
    for i in range(len(team_np)-window_size):        
      row = [a for a in team_np[i:i+window_size]]
      X_list.append(row)

      label = team_label[i+window_size]
      y_list.append(label)

X_list = np.array(X_list)
y_list = np.array(y_list)

X_list.shape, y_list.shape

((23422, 10, 34), (23422,))

In [26]:
from model import ModelConfig

In [27]:
model_config = ModelConfig(X_list,y_list,[.75,.10,.15])

In [28]:
X_train,y_train,X_test,y_test,X_val,y_val = model_config.train_test_val()

In [29]:
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import RootMeanSquaredError

model = model_config.create_sequential()
model.add(LSTM(64))
model.add(Dense(8,'relu'))
model.add(Dense(1,'sigmoid'))

model.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=.00001), metrics=[RootMeanSquaredError()])

In [33]:
model_config.create_config('test','/home/alexp/NBA_Models/models/model_configs/')

'Config Created: /home/alexp/NBA_Models/models/model_configs/testv3.json'

In [31]:
model.metrics

[]

In [32]:
tf.config.list_physical_devices('GPU')

NameError: name 'tf' is not defined

In [ ]:
model.optimizer